# Advanced Querying Mongo

Importing libraries and setting up connection

In [38]:
import pandas as pd
from pymongo import MongoClient
client = MongoClient("localhost:27017")

In [7]:
db.list_collection_names()

['restaurants', 'companies']

In [9]:
coll = db.get_collection("companies")
coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

In [12]:
result = coll.find_one()
result.keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [42]:
result = pd.DataFrame(coll.find({"name": "Babelgum"}, {"name":1, "_id":0}))
result

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [109]:
result2 = pd.DataFrame(coll.find({"number_of_employees":{"$gt":5000}}).limit(20).sort("number_of_employees", -1)).head(5)
result2

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,investments,acquisition,acquisitions,offices,milestones,ipo,video_embeds,screenshots,external_links,partners
0,52cdef7d4bab8bd67529941a,Siemens,siemens,http://www.crunchbase.com/company/siemens,http://www.siemens.com,,,Siemens,hardware,405000,...,[],None,"[{'price_amount': 418000000, 'price_currency_c...","[{'description': '', 'address1': 'Wittelsbache...","[{'id': 35852, 'description': 'Siemens Receive...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<iframe width=""420"" height=""3...","[{'available_sizes': [[[150, 92], 'assets/imag...",[],[]
1,52cdef7c4bab8bd67529856a,IBM,ibm,http://www.crunchbase.com/company/ibm,http://www.ibm.com,,,IBM,software,388000,...,"[{'funding_round': {'round_code': 'c', 'source...",None,"[{'price_amount': None, 'price_currency_code':...","[{'description': 'Corporate Headquarters', 'ad...","[{'id': 10471, 'description': 'IBM Completes A...","{'valuation_amount': None, 'valuation_currency...","[{'embed_code': '<embed src=""http://blip.tv/pl...","[{'available_sizes': [[[150, 93], 'assets/imag...","[{'external_url': 'http://mashpedia.com/IBM', ...",[]
2,52cdef7d4bab8bd675299d33,Toyota,toyota,http://www.crunchbase.com/company/toyota,http://www.toyota-global.com,,,Toyota,enterprise,320000,...,"[{'funding_round': {'round_code': 'a', 'source...",None,[],"[{'description': 'HQ', 'address1': 'Toyota Mot...",[],None,"[{'embed_code': '<iframe width=""430"" height=""3...","[{'available_sizes': [[[150, 84], 'assets/imag...",[],[]
3,52cdef7c4bab8bd675297e89,PayPal,paypal,http://www.crunchbase.com/company/paypal,http://www.paypal.com,,,paypal,finance,300000,...,"[{'funding_round': {'round_code': 'c', 'source...","{'price_amount': 1500000000, 'price_currency_c...","[{'price_amount': None, 'price_currency_code':...","[{'description': '', 'address1': '2145 E Hamil...","[{'id': 11916, 'description': 'Reserve Bank Of...",None,[],"[{'available_sizes': [[[150, 120], 'assets/ima...",[{'external_url': 'http://www.sociableblog.com...,[]
4,52cdef7e4bab8bd67529b0fe,Nippon Telegraph and Telephone Corporation,nippon-telegraph-and-telephone-corporation,http://www.crunchbase.com/company/nippon-teleg...,http://www.ntt.co.jp/index_e.html,,,,None,227000,...,"[{'funding_round': {'round_code': 'e', 'source...",None,[],"[{'description': 'NTT', 'address1': '3-1, Otem...",[],None,[],"[{'available_sizes': [[[150, 42], 'assets/imag...",[],[]


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [51]:
result3 = pd.DataFrame(coll.find({"founded_year":{"$gte":2000, "$lte":2005}}, {"name":1, "founded_year":1, "_id":0}))
result3

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005
...,...,...
3729,EnterSys Group,2000
3730,Axon Solutions,2004
3731,Intergy,2003
3732,AfterLogic,2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [108]:
result4 = pd.DataFrame(coll.find({"$and":
                                [{"ipo.valuation_amount":{"$gt":100000000}},
                                 {"founded_year":{"$lt":2010}}
                                ]},
                                 {"name":1, "ipo.valuation_amount":1, "_id":0}).sort("ipo.valuation_amount", -1)).head(5)
result4

,name,ipo
0,GREE,{'valuation_amount': 108960000000}
1,Facebook,{'valuation_amount': 104000000000}
2,Amazon,{'valuation_amount': 100000000000}
3,Twitter,{'valuation_amount': 18100000000}
4,Groupon,{'valuation_amount': 12800000000}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [97]:
result5 = pd.DataFrame(coll.find({"$and":
                                [{"number_of_employees":{"$lt":1000}},
                                 {"founded_year":{"$lt":2005}}
                                ]}, {"name":1, "number_of_employees":1, "founded_year":1, "_id":0}
                                  
                                ).sort("number_of_employees",-1).limit(10))
result5

,name,number_of_employees,founded_year
0,Infinera Corporation,974,2000
1,NorthPoint Communications Group,948,1997
2,888 Holdings,931,1997
3,Forrester Research,903,1983
4,SonicWALL,900,1991
5,Webmetrics,900,1999
6,Cornerstone OnDemand,881,1999
7,Mozilla,800,1998
8,Buongiorno,800,1999
9,Yelp,800,2004


### 6. All the companies that don't include the `partners` field.

In [102]:
result6 = list(coll.find({"partners": {"$exists":False}}))
print(result6)
print("No company")

[]
No company


### 7. All the companies that have a null type of value on the `category_code` field.

In [107]:
result7 = pd.DataFrame(coll.find({"category_code":{"$type": "null"}}, {"name":1, "category_code":1})).head(5)
result7

,_id,name,category_code
0,52cdef7c4bab8bd6752980f6,Collective,None
1,52cdef7c4bab8bd675298225,Snimmer,None
2,52cdef7c4bab8bd675298226,KoolIM,None
3,52cdef7c4bab8bd675298261,Level9 Media,None
4,52cdef7c4bab8bd675298262,VidKing,None


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [112]:
result8 = pd.DataFrame(coll.find({"$and":
                                [{"number_of_employees":{"$lt":1000}},
                                 {"number_of_employees":{"$gte":100}}
                                ]}, {"name":1, "number_of_employees":1, "_id":0}
                                  
                                ).sort("number_of_employees",-1))
result8

,name,number_of_employees
0,Datamonitor,984
1,Infinera Corporation,974
2,Box,950
3,NorthPoint Communications Group,948
4,888 Holdings,931
...,...,...
912,ZoomSystems,100
913,Exent,100
914,Mashable,100
915,Applied Language Solutions,100


### 9. Order all the companies by their IPO price in a descending order.

In [128]:
result9 = pd.DataFrame(coll.find({}, {"name":1, "ipo.valuation_amount":1, "_id":0}).sort("ipo.valuation_amount",-1)).head(5)
result9

,name,ipo
0,GREE,{'valuation_amount': 108960000000}
1,Facebook,{'valuation_amount': 104000000000}
2,Amazon,{'valuation_amount': 100000000000}
3,Twitter,{'valuation_amount': 18100000000}
4,Groupon,{'valuation_amount': 12800000000}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [132]:
result10 = pd.DataFrame(coll.find({}, {"name":1, "number_of_employees":1, "_id":0}).sort("number_of_employees",-1).limit(10))
result10

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [137]:
result11 = pd.DataFrame(coll.find({"founded_month":{"$gt":6}},                                  
                                  {"name":1, "founded_month":1, "_id":0}
                                 ).sort("founded_month", -1).limit(1000))
result11

,name,founded_month
0,Glogster,12
1,Microworks,12
2,Claremontian,12
3,Vivity Labs,12
4,Gengo,12
...,...,...
995,Why Hasn't He?,10
996,Fresho,10
997,Driftr,10
998,Radha Krishna Portal,10


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [142]:
result12 = pd.DataFrame(coll.find({"$and":
                        [{"founded_year":{"$lt":2000}},
                        {"acquisition.price_amount":{"$gt":10000000}}]
                        },
                        {"name":1, "acquisition.price_amount":1, "_id":0}
                        ).sort("acquisition.price_amount",-1)).head(5)
result12

,name,acquisition
0,BEA Systems,{'price_amount': 8500000000}
1,Navteq,{'price_amount': 8100000000}
2,Sun Microsystems,{'price_amount': 7400000000}
3,Pixar,{'price_amount': 7400000000}
4,LSI,{'price_amount': 6600000000}


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [147]:
result13 = pd.DataFrame(coll.find({"acquisition.acquired_year":{"$gt":2010}},
                                  {"name":1, "acquisition":1, "_id":0}
                        ).sort("acquisition.price_amount")).head(5)
result13

,name,acquisition
0,Geni,"{'price_amount': None, 'price_currency_code': ..."
1,Kyte,"{'price_amount': None, 'price_currency_code': ..."
2,blogTV,"{'price_amount': None, 'price_currency_code': ..."
3,delicious,"{'price_amount': None, 'price_currency_code': ..."
4,Mashery,"{'price_amount': None, 'price_currency_code': ..."


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [152]:
result14 = pd.DataFrame(coll.find({}, {"name":1, "founded_year":1, "_id":0}).sort("founded_year",-1)).head(5)
result14

,name,founded_year
0,Fixya,2013.0
1,Wamba,2013.0
2,Advaliant,2013.0
3,Fluc,2013.0
4,iBazar,2013.0


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [156]:
result15 = pd.DataFrame(coll.find({"$and":
                        [{"founded_day":{"$gte":1}},
                         {"founded_day":{"$lte":7}}
                        ]},
                        {"name":1, "acquisition.price_amount":1, "_id":0}
                        ).sort("acquisition.price_amount",-1).limit(10))
result15

,name,acquisition
0,Netscape,{'price_amount': 4200000000}
1,PayPal,{'price_amount': 1500000000}
2,Zappos,{'price_amount': 1200000000}
3,Alibaba,{'price_amount': 1000000000}
4,Postini,{'price_amount': 625000000}
5,Danger,{'price_amount': 500000000}
6,Clearwell Systems,{'price_amount': 410000000}
7,PrimeSense,{'price_amount': 345000000}
8,Amobee,{'price_amount': 321000000}
9,BlueLithium,{'price_amount': 300000000}


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [162]:
result16 = pd.DataFrame(coll.find({"$and":
                                    [{"number_of_employees":{"$gt":4000}},
                                     {"category_code":"web"}]},
                                  {"name":1, "category_code":1, "number_of_employees":1, "_id":0})
                                  .sort("number_of_employees",1))
result16

,name,category_code,number_of_employees
0,Expedia,web,4400
1,AOL,web,8000
2,Webkinz,web,8657
3,Rakuten,web,10000
4,Los Angeles Times Media Group,web,10000
5,Groupon,web,10000
6,Yahoo!,web,13600
7,eBay,web,15000
8,Experian,web,15500


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [167]:
result17 = pd.DataFrame(coll.find({"$and":
                                 [{"acquisition.price_amount":{"$gt":10000000}},
                                  {"acquisition.price_currency_code":"EUR"}]
                                 },
                                 {"name":1, "acquisition.price_amount":1, "acquisition.price_currency_code": 1, "_id":0})
                                 .sort("acquisition.price_amount", -1))
result17

,name,acquisition
0,ZYB,"{'price_amount': 31500000, 'price_currency_cod..."
1,Apertio,"{'price_amount': 140000000, 'price_currency_co..."
2,Greenfield Online,"{'price_amount': 40000000, 'price_currency_cod..."
3,Webedia,"{'price_amount': 70000000, 'price_currency_cod..."
4,Wayfinder,"{'price_amount': 24000000, 'price_currency_cod..."
5,Tuenti Technologies,"{'price_amount': 70000000, 'price_currency_cod..."
6,BioMed Central,"{'price_amount': 43400000, 'price_currency_cod..."


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [168]:
result18 = pd.DataFrame(coll.find({"$and":
                        [{"founded_month":{"$gte":1}},
                         {"founded_month":{"$lte":3}}
                        ]},
                        {"name":1, "acquisition":1, "_id":0}
                        ).limit(10))
result18

,name,acquisition
0,Facebook,None
1,Twitter,None
2,StumbleUpon,"{'price_amount': 29000000, 'price_currency_cod..."
3,Gizmoz,"{'price_amount': None, 'price_currency_code': ..."
4,Helio,"{'price_amount': 39000000, 'price_currency_cod..."
5,Babelgum,None
6,Yahoo!,None
7,Mahalo,None
8,Meetup,None
9,Jangl SMS,None


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [173]:
result19 = pd.DataFrame(coll.find({"$and":
                        [{"founded_year":{"$gt":2000}},
                         {"founded_year":{"$lt":2010}},
                         {"acquisition.acquired_year":{"$gte":2011}}
                        ]},
                        {"name":1, "acquisition":1, "founded_year":1, "_id":0}
                        ).sort("founded_year",-1).limit(10))
result19

,name,founded_year,acquisition
0,TweetPhoto,2009,"{'price_amount': None, 'price_currency_code': ..."
1,oneforty,2009,"{'price_amount': None, 'price_currency_code': ..."
2,5to1,2009,"{'price_amount': 30000000, 'price_currency_cod..."
3,Trunkt,2009,"{'price_amount': None, 'price_currency_code': ..."
4,Shutl,2009,"{'price_amount': None, 'price_currency_code': ..."
5,Infochimps,2009,"{'price_amount': None, 'price_currency_code': ..."
6,1000 Markets,2009,"{'price_amount': None, 'price_currency_code': ..."
7,Wefollow,2009,"{'price_amount': None, 'price_currency_code': ..."
8,Honk,2009,"{'price_amount': None, 'price_currency_code': ..."
9,HyperWeek,2009,"{'price_amount': None, 'price_currency_code': ..."


### 20. All the companies that have been 'deadpooled' after the third year.

In [280]:
result20 = pd.DataFrame(coll.find({"deadpooled_year":{"$gt":3}}, 
                                  {"name":1, "founded_year":1, "deadpooled_year":1, "_id":0}))

In [281]:
result20.dropna(subset=["founded_year"], axis = 0, inplace=True)

In [282]:
result20["founded_year"] = result20["founded_year"].astype(int)

In [283]:
result20["difference"] = result20["deadpooled_year"]-result20["founded_year"]

In [284]:
for i in list(result20["difference"]):
    if abs(i) <= 3:
        result20 = result20.loc[result20["difference"] > 3]
        result20.reset_index(drop=True)

In [285]:
result20

,name,founded_year,deadpooled_year,difference
1,Babelgum,2007,2013,6
3,Thoof,2006,2013,7
5,Wesabe,2005,2010,5
7,Stickam,2006,2013,7
8,AllPeers,2004,2008,4
...,...,...,...,...
908,SpeakSoft,2007,2012,5
911,Tagito,2008,2012,4
914,Nordic Windpower,2007,2013,6
919,Nethra Imaging,2003,2012,9


In [287]:
print(f"Total companies: {result20.shape[0]}")

Total companies: 437


I did it with pandas :) cheating?